In [1]:
import time
import os
import shutil
import sys
import h5py
import tensorflow                             as tf
import numpy                                  as np
import seaborn                                as sns
import pandas                                 as pd
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path
import pysftp                                 as sftp
import random                                 as rnd
from matplotlib                 import pyplot as plt 

print("="*50)
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.4.1
 Eager execution: True


In [2]:
def write_predictiondata(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec):
    
    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]
                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics


In [3]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT_MAITH/surqct/'

RatesType           = 'KExcit'

# ExcitType           = 'KInel'
# NNRunIdx            = 9
ExcitType           = 'KExch'
NNRunIdx            = 7

PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures_nondim/' + ExcitType + '/' 

TTranVec            = [10000.0]

# Molecules           = ['N2','N2']
# Atoms               = ['N','N']
# System = 'N3_NASA'

# Molecules           = ['NO','NO']
# Atoms               = ['N','N']
# System = 'NON_UMN'

Molecules           = ['CO','O2']
Atoms               = ['O','C']
System = 'CO2_NASA'

ZeroDRunIdx         = NNRunIdx

MinValueTrain       = 1.e-15

In [4]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...


In [5]:
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld 
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 
InputData.KineticFldr     = InputData.WORKSPACE_PATH+'/Air_Database/Run_0D_surQCT/database/kinetics/nondim_'+System+'_Active_Run'+str(ZeroDRunIdx)+'/'

# ## N2
# InputData.PathToLevelsFile = [WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv',
#                               WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv']
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/N3_NASA.hdf5'
# InputData.Molecules       = ['N2','N2'] 
# InputData.PathToDiatFile  = [WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp',
#                              WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp']   
# #InputData.PathToGrouping  = self.WORKSPACE_PATH  + '/Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_DPM45.csv'

# # NO
# InputData.PathToLevelsFile = [WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/NO_UMN_Bottom_Vib_we_nd.csv',
#                               WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/NO_UMN_Bottom_Vib_we_nd.csv']
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/NON_UMN.hdf5'
# InputData.Molecules       = ['NO','NO'] 
# InputData.PathToDiatFile  = [WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/NO/UMN/Recomputed.inp',
#                              WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/NO/UMN/Recomputed.inp']   

# # NO, N2
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/NON_UMN.hdf5'
# InputData.Molecules       = ['NO','N2'] 
# InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/NO_UMN_log_nd.csv',
#                              InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_UMN_ForN2O2_log_nd.csv']
# InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/NO/UMN/Recomputed.inp',
#                              InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/UMN_ForN2O2/Recomputed.inp'] 

# # For CO from COO PES
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/CO2_NASA.hdf5'
# InputData.Molecules       = ['CO','CO'] 
# InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/CO_NASA_log_nd.csv',
#                              InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/CO_NASA_log_nd.csv']
# InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_NASA.dat',
#                              InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_NASA.dat']

## For CO from COO PES
InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/CO2_NASA.hdf5'
InputData.Molecules       = ['CO','O2'] 
InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/CO_NASA_Diss_log_nd_O2RefForHeteroExch.csv',
                             InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/O2_NASA_Diss_log_nd_O2RefForHeteroExch.csv']
InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_NASA.dat',
                             InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/O2/NASA/O2_levels_venturi_new.dat']



[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [6]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg = 0
InputData.TrainIntFlg       = 0
NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'ri_i', 'log_rorMin_i', 'TTran_i', 'EVib', 'ERot', 'ri', 'log_rorMin', 'TTran']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /home/venturi/WORKSPACE//SurQCT_MAITH/surqct//../KExcit/all_temperatures_nondim/KExch//Run_7/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /home/venturi/WORKSPACE//SurQCT_MAITH/surqct//../KExcit/all_temperatures_nondim/KExch//Run_7/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /home/venturi/WORKSPACE//SurQCT_MAITH/surqct//../KExcit/all_temperatures_nondim/KExch//Run_7/Params//ModelCheckpoint/cp-0845.ckpt


In [7]:
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
# MinValueTrain      = 1.e-16 * InputData.MultFact
# MinValueTest       = 1.e-16 * InputData.MultFact
# NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

InputData.iLevelsVecTest = list(np.array(InputData.iLevelsVecTest) - 1)

[SurQCT]:   Reading Variables:  ['jqn', 'ERot', 'EVib', 'log_rorMin', 'ri', 'vqn']


In [8]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array(TTranVec), np.array(TTranVec))
    DiatData.append(DiatDataTemp)

    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)

[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/CO_NASA_Diss_log_nd_O2RefForHeteroExch.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_NASA.dat
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/O2_NASA_Diss_log_nd_O2RefForHeteroExch.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/NASA/O2_levels_venturi_new.dat


In [9]:
if (Molecules[0] == 'N2'):
    
    vqnVec = DiatData[0]['vqn'].to_numpy(int)
    jqnVec = DiatData[0]['jqn'].to_numpy(int)

    QnsToLevel = np.zeros((vqnVec.max()+1, jqnVec.max()+1), dtype=int)
    for iLevel in range(NLevels[0]):
        QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]] = iLevel
    Maxjqn = np.argmax(QnsToLevel, axis=1)  # Max J value for each vibrational quantum number

    # P and M is for Plus and Minus
    EvenToOddP = np.zeros(NLevels[0], int)
    EvenToOddM = np.zeros(NLevels[0], int)
    OddToEvenP = np.zeros(NLevels[0], int)
    OddToEvenM = np.zeros(NLevels[0], int) 
    for iLevel in range(NLevels[0]):

        if (np.mod(jqnVec[iLevel],2) == 0):

            if (jqnVec[iLevel]+1 <= Maxjqn[vqnVec[iLevel]]):
                EvenToOddP[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]+1]
            else:
                EvenToOddP[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]-1]

            if (jqnVec[iLevel]-1 >= 0):
                EvenToOddM[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]-1]
            else:
                EvenToOddM[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]+1]

        else:

            if (jqnVec[iLevel]+1 <= Maxjqn[vqnVec[iLevel]]):
                OddToEvenP[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]+1]
            else:
                OddToEvenP[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]-1]

            if (jqnVec[iLevel]-1 >= 0):
                OddToEvenM[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]-1]
            else:
                OddToEvenM[iLevel] = QnsToLevel[vqnVec[iLevel], jqnVec[iLevel]+1]

# N2 Prediction

In [10]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_OddEvenJAdded.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_OddEvenJAdded.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            KExcit_NN_FWD          = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
  
        ## BWD Rates
        iiIdxVec              = jIdxVecNo
        jIdxVec_BWD           = [iIdx]*jNLevelsNo
        
        TTranVecTemp          = np.ones((jNLevelsNo))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = iiIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = iiIdxVec
    
        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec_BWD,:].copy() 
        jLevelsDataTemp.index = iiIdxVec
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec_BWD,:].copy()        
        kLevelsDataTemp.index = iiIdxVec
    
        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    
        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTempp_BWD            = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        if (len(xTempp_BWD[NN_KExcit.xTrainingVar]) > 0):
            KExcit_NN_BWD          = np.exp( NN_KExcit.Model.predict(xTempp_BWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
            KExcit_NN_BWD[:,0]     = KExcit_NN_BWD[:,0] / RatioNo
    
    
            iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            iLevelsData.columns   = [(VarName + '_j') for VarName in iLevelsData.columns]
    
            jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
            jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    
            kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            kLevelsData.columns   = [(VarName + '_i') for VarName in kLevelsData.columns]
    
            xTemp_BWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
    
    
            
        ### Concatenating
            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                xTemp     = pd.concat([xTemp_FWD, xTemp_BWD], axis=0)
                KExcit_NN = np.concatenate((KExcit_NN_FWD, KExcit_NN_BWD), axis=0)
            else:
                xTemp     = xTemp_BWD
                KExcit_NN = KExcit_NN_BWD
        else:
            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                xTemp     = xTemp_FWD
                KExcit_NN = KExcit_NN_FWD
          
        
        if (InputData.Molecules[0] == 'N2'):
            MaskRuleYES = np.mod(jqnVec[iIdx],2) == np.mod(jqnVec,2)
            MaskRuleNO  = np.mod(jqnVec[iIdx],2) != np.mod(jqnVec,2)
            if (np.mod(jqnVec[iIdx],2) == 0):
                KExcit_NN[OddToEvenP[MaskRuleNO]] += KExcit_NN[MaskRuleNO]/2.
                KExcit_NN[OddToEvenM[MaskRuleNO]] += KExcit_NN[MaskRuleNO]/2.
            else:
                KExcit_NN[EvenToOddP[MaskRuleNO]] += KExcit_NN[MaskRuleNO]/2.
                KExcit_NN[EvenToOddM[MaskRuleNO]] += KExcit_NN[MaskRuleNO]/2.
            
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
        
            if(ExcitType == 'KInel'): 
                KInel_NN1 = KExcit_NN[jIdxVec]
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN1)
        
            elif(ExcitType == 'KExch'):      
                KExch_NN1 = KExcit_NN[jIdxVec]
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN1)
                
        KExcit_NN[MaskRuleNO] = 0.
                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)

    


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/9399 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run4//T10000K/Inel_OddEvenJAdded.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 9399/9399 [59:38<00:00,  2.63it/s] 

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run4//T10000K/Inel_OddEvenJAdded.dat


# NO Prediction: Inel and Homogenous Exch

In [9]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type2.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            if(ExcitType == 'KInel'):   
                KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
            
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 1/6739 [00:00<21:15,  5.28it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_NON_UMN_Active_Run15//T10000K/Exch_Type2.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 6739/6739 [26:44<00:00,  4.20it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_NON_UMN_Active_Run15//T10000K/Exch_Type2.dat


# NO Heterogenous Exchange 

In [12]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        jIdxVec               = [jIdx for jIdx in np.arange(NLevels[1])]
        jNLevels              = len(jIdxVec)

        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            if(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/6739 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_NON_UMN_Active_Run4//T10000K/Exch_Type1.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 6739/6739 [19:06<00:00,  5.88it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_NON_UMN_Active_Run4//T10000K/Exch_Type1.dat


# CO Inel 

In [ ]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)
        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            if(ExcitType == 'KInel'):   
                KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
            
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:    TTran =  7500.0


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/13521 [00:00<?, ?it/s]

[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_CO2_NASA_Active_Run6//T7500K/Exch_Type1.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   1%|▏         | 173/13521 [01:00<1:22:16,  2.70it/s]

# Heterogenous CO

In [9]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass


for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)


    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type2.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        jIdxVec               = [jIdx for jIdx in np.arange(NLevels[1])]
        jNLevels              = len(jIdxVec)

        
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
        
        TTranVecTemp          = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVecTemp})
        TTranDataTemp.index   = jIdxVec

        
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
        
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

        
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
        
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
        
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
        
        
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            if(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)

                    
    if(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)



[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/13521 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_CO2_NASA_Active_Run7//T10000K/Exch_Type2.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 13521/13521 [41:36<00:00,  5.42it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_CO2_NASA_Active_Run7//T10000K/Exch_Type2.dat
